<h1><font size=12>
Weather Derivatites </h1>
<h1> Precipitation Full Dataset Builder <br></h1>

Developed by [Jesus Solano](mailto:ja.solano588@uniandes.edu.co) <br>  
22 September 2018 


In [1]:
# Import needed libraries. 
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
import time
from io import StringIO
import datetime 
import pickle

# Import Datasets

In [2]:
#### EL Dorado Airport Precipitation.

# Configure path to read txts.

path = '../datasets/'

# Download the update dataset.

import os

if not os.path.exists(path+'precipitationAllTime.csv'):

  ! wget https://github.com/jesugome/WeatherDerivates/raw/master/datasets/precipitationAllTime.csv -P path

  ! wget https://github.com/jesugome/WeatherDerivates/raw/master/datasets/precipitationRemovingAverageAllTime.csv -P path


precipitationAllTime = pd.read_csv(path+'precipitationAllTime.csv',header=None, names = ['Date','Prep'])

precipitationReAllTime = pd.read_csv(path+'precipitationRemovingAverageAllTime.csv')

precipitationAllTime['Date'] = pd.to_datetime(precipitationAllTime['Date'])
precipitationAllTime = precipitationAllTime.set_index('Date')

print('* Note: Precipitations are in mmm(milimetros de agua)--- 1mm = 1L/m²')
precipitationAllTime.tail()



* Note: Precipitations are in mmm(milimetros de agua)--- 1mm = 1L/m²


,Prep
Date,
2015-12-27,0.0
2015-12-28,0.0
2015-12-29,0.0
2015-12-30,0.0
2015-12-31,0.0


In [3]:
#### Daily Nino3.4 index.

# Configure path to read txts.

path = '../datasets/'

# Download the update dataset.

import os

if not os.path.exists(path+'nino34_daily.dat'):

  ! wget https://github.com/jesugome/WeatherDerivates/raw/master/datasets/nino34_daily.dat -P path
    
#tempFile = StringIO(''.join(l.replace('  ', ' ') for l in open(path+'nino34_daily.dat')))

nino34AllTime = pd.read_csv(StringIO(''.join(l.replace('   ', '  ').replace('  ',' ') for l in open(path+'nino34_daily.dat'))),header=None, names = ['Date','nino34'], skiprows= lambda x: x in range(0,100), sep= ' ')

nino34AllTime['Date'] = nino34AllTime['Date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'))

nino34AllTime = nino34AllTime.set_index('Date')

nino34AllTime.tail()


,nino34
Date,
2018-07-28,0.314189
2018-07-29,0.165250
2018-07-30,0.117431
2018-07-31,0.189321
2018-08-01,0.211368


In [4]:
### ENSO probabilistic forecast.

# Open saved data.
ensoForecast = pickle.load(open('../datasets/ensoForecastProb/ensoForecastProbabilities.pickle','rb'))

# Print an example .. ( Format needed)

ensoForecast['2005-01']

,Season,La Niña,Neutral,El Niño
0,JFM 2005,0%,15%,85%
1,FMA 2005,0%,17%,83%
2,MAM 2005,1%,24%,75%
3,AMJ 2005,2%,43%,55%
4,MJJ 2005,4%,51%,45%
5,JJA 2005,6%,54%,40%
6,JAS 2005,8%,52%,40%
7,ASO 2005,9%,51%,40%
8,SON 2005,10%,50%,40%
9,OND 2005,10%,50%,40%


# Create Total Dataframe

In [10]:
### Create total dataframe.

# Set variables names.
totalDataframeColumns = ['Prep','Month','nino34','probNeutral','probNino','probNina','state', 'nextState']

# Create dataframe.

allDataDataframe = pd.DataFrame(columns=totalDataframeColumns) 

# Fill datafame information.

# Generate a timestamp with all days in simulation.
startDate = '2005-01-01'
endDate  = '2015-12-31'

dates = pd.date_range(startDate, endDate, freq='D')

for date in dates: 
    
    # Fill precipitation amount.
    allDataDataframe.loc[date.strftime('%Y-%m-%d'),'Prep'] = precipitationAllTime.loc[date.strftime('%Y-%m-%d'),'Prep']
    
    # Fill month of date 
    allDataDataframe.loc[date.strftime('%Y-%m-%d'),'Month'] = date.month
    
    # Fill daily nino 34.
    allDataDataframe.loc[date.strftime('%Y-%m-%d'),'nino34'] = nino34AllTime.loc[date.strftime('%Y-%m-%d'),'nino34']
    
    # Fill neutral Enso forecast probability.
    allDataDataframe.loc[date.strftime('%Y-%m-%d'),'probNeutral'] = float(ensoForecast[date.strftime('%Y-%m')].loc[0,'Neutral'].strip('%').strip('~'))/100
    
    # Fill El Nino ENSO forecast probability. 
    allDataDataframe.loc[date.strftime('%Y-%m-%d'),'probNino'] = float(ensoForecast[date.strftime('%Y-%m')].loc[0,'El Niño'].strip('%').strip('~'))/100
    
    # Fill La Nina ENSO forecast probability. 
    allDataDataframe.loc[date.strftime('%Y-%m-%d'),'probNina'] = float(ensoForecast[date.strftime('%Y-%m')].loc[0,'La Niña'].strip('%').strip('~'))/100
    
    # Fill State. 
    allDataDataframe.loc[date.strftime('%Y-%m-%d'),'state'] = (1 if precipitationAllTime.loc[date.strftime('%Y-%m-%d'),'Prep']>0 else 0 )
    
    # Fill next State.
    nextDate = date + 1
    
    if date != dates[-1]:
        allDataDataframe.loc[date.strftime('%Y-%m-%d'),'nextState'] = (1 if precipitationAllTime.loc[nextDate.strftime('%Y-%m-%d'),'Prep']>0 else 0 )

        
allDataDataframe = allDataDataframe[:-1]

In [11]:
allDataDataframe.head(150)

,Prep,Month,nino34,probNeutral,probNino,probNina,state,nextState
2005-01-01,0,1,0.606186,0.15,0.85,0,0,0
2005-01-02,0,1,0.599358,0.15,0.85,0,0,0
2005-01-03,0,1,0.646784,0.15,0.85,0,0,1
2005-01-04,1.6,1,0.663696,0.15,0.85,0,1,0
2005-01-05,0,1,0.71133,0.15,0.85,0,0,0
2005-01-06,0,1,0.679185,0.15,0.85,0,0,0
2005-01-07,0,1,0.558135,0.15,0.85,0,0,0
2005-01-08,0,1,0.451021,0.15,0.85,0,0,0
2005-01-09,0,1,0.593434,0.15,0.85,0,0,1
2005-01-10,0.4,1,0.672419,0.15,0.85,0,1,0


# Export Ideam Data to external Files

In [13]:
# Save total data to .csv

allDataDataframe.to_csv('../datasets/fullDataset/completeDailyDataset.csv')

pickle.dump(allDataDataframe, open(path+'/fullDataset/completeDailyDataset.pickle','wb'))